In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from data_loader import load_sample_data, preprocess_data
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load and preprocess data
stations_info, status_sample = load_sample_data()
merged_data = preprocess_data(stations_info, status_sample)

# Function to perform time series analysis for a given station
def perform_time_series_analysis(data, station_id, start_date, end_date):
    station_data = data[(data['station_id'] == station_id) & 
                        (data['last_reported'] >= start_date) & 
                        (data['last_reported'] <= end_date)]
    station_data = station_data.set_index('last_reported')['num_bikes_available'].resample('H').mean()
    
    # Fit ARIMA model
    model = ARIMA(station_data, order=(1, 1, 1))
    results = model.fit()
    
    # Make predictions
    forecast = results.forecast(steps=24)
    
    return station_data, forecast, results

# Function to perform linear regression for a given station
def perform_linear_regression(data, station_id, start_date, end_date):
    station_data = data[(data['station_id'] == station_id) & 
                        (data['last_reported'] >= start_date) & 
                        (data['last_reported'] <= end_date)]
    
    X = station_data[['hour', 'day_of_week']]
    y = station_data['num_bikes_available']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model, X_test, y_test, y_pred, mse, r2

# Interactive time series plot
def plot_interactive_time_series(station_data, forecast, station_id):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=station_data.index, y=station_data.values, mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values, mode='lines', name='Forecast'))
    
    fig.update_layout(title=f'Time Series Analysis for Station {station_id}',
                      xaxis_title='Date',
                      yaxis_title='Number of Available Bikes')
    
    return fig

# Interactive linear regression plot
def plot_interactive_linear_regression(X_test, y_test, y_pred, station_id):
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', name='Predictions'))
    fig.add_trace(go.Scatter(x=[y_test.min(), y_test.max()], y=[y_test.min(), y_test.max()], 
                             mode='lines', name='Perfect Prediction'))
    
    fig.update_layout(title=f'Linear Regression Results for Station {station_id}',
                      xaxis_title='Actual',
                      yaxis_title='Predicted')
    
    return fig

# Heatmap of station popularity
def plot_station_popularity_heatmap(data):
    station_popularity = data.groupby('station_id')['num_bikes_available'].mean().reset_index()
    station_popularity = pd.merge(station_popularity, stations_info[['station_id', 'lat', 'lon']], on='station_id')
    
    fig = px.density_mapbox(station_popularity, lat='lat', lon='lon', z='num_bikes_available', radius=10,
                            center=dict(lat=41.3851, lon=2.1734), zoom=12,
                            mapbox_style="stamen-terrain")
    
    fig.update_layout(title='Station Popularity Heatmap')
    
    return fig

# Compare stations by neighborhood
def compare_stations_by_neighborhood(data):
    station_data = data.groupby(['station_id', 'hour'])['num_bikes_available'].mean().reset_index()
    
    fig = px.line(station_data, x='hour', y='num_bikes_available', color='station_id',
                  title='Average Bike Availability by Station')
    
    fig.update_layout(legend_title_text='Station ID')
    return fig

# Main analysis function
def perform_advanced_analysis(data, start_date, end_date):
      # Check for required columns
    required_columns = ['station_id', 'num_bikes_available', 'last_reported', 'hour', 'day_of_week', 'lat', 'lon']
    missing_columns = [col for col in required_columns if col not in data.columns]
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")

    # Select a sample station for demonstration
    station_id = data['station_id'].iloc[0]
    
    # Time Series Analysis
    station_data, forecast, results = perform_time_series_analysis(data, station_id, start_date, end_date)
    ts_plot = plot_interactive_time_series(station_data, forecast, station_id)
    ts_plot.show()
    
    # Linear Regression
    model, X_test, y_test, y_pred, mse, r2 = perform_linear_regression(data, station_id, start_date, end_date)
    lr_plot = plot_interactive_linear_regression(X_test, y_test, y_pred, station_id)
    lr_plot.show()
    
    print(f"Linear Regression Results for Station {station_id}:")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R-squared: {r2:.2f}")
    
    # Station Popularity Heatmap
    heatmap = plot_station_popularity_heatmap(data)
    heatmap.show()
    
    # Neighborhood Comparison
    neighborhood_plot = compare_stations_by_neighborhood(data)
    neighborhood_plot.show()

# Run the analysis
start_date = pd.to_datetime('2024-09-01')
end_date = pd.to_datetime('2024-09-02')
perform_advanced_analysis(merged_data, start_date, end_date)

INFO:data_loader:Successfully connected to MongoDB
INFO:data_loader:Loaded 516 stations
INFO:data_loader:Attempting to load status data for one day in September 2024
INFO:data_loader:Loaded 146930 status records
INFO:data_loader:Successfully loaded and sampled data for one day in September 2024
INFO:data_loader:Starting data preprocessing
INFO:data_loader:Merged data shape: (14693, 12)
INFO:data_loader:Data shape after filtering non-existent stations: (14693, 12)
INFO:data_loader:Data preprocessing completed
C:\Users\danys\AppData\Local\Temp\ipykernel_29188\1336536625.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  station_data = station_data.set_index('last_reported')['num_bikes_available'].resample('H').mean()


Linear Regression Results for Station 88:
Mean Squared Error: 20.27
R-squared: 0.02
